# Device Failure Project - Sujal Padhiyar -Part 3

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.model_selection import train_test_split as tts #training and testing data split
from sklearn import metrics #accuracy measure

In [2]:
df=pd.read_csv('dataprep.csv')

In [3]:
df=df.drop(['Unnamed: 0'],axis=1)

In [4]:
df.head()

,failure,attribute4,attribute5,work_days,device_type,att2,att1,att39,att78,month,total_mile
0,0,52,6,6,0,7,26953834,7,0,1,1966
1,0,0,5,6,0,0,30418260,0,0,5,209
2,0,0,6,6,0,0,28042412,3,0,1,1612
3,0,0,12,48,0,0,30032246,0,0,2,22097
4,0,0,6,6,0,0,29445357,0,0,1,1965


In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import KFold

In [6]:
prep=df.copy()

In [7]:
prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168 entries, 0 to 1167
Data columns (total 11 columns):
failure        1168 non-null int64
attribute4     1168 non-null int64
attribute5     1168 non-null int64
work_days      1168 non-null int64
device_type    1168 non-null int64
att2           1168 non-null int64
att1           1168 non-null int64
att39          1168 non-null int64
att78          1168 non-null int64
month          1168 non-null int64
total_mile     1168 non-null int64
dtypes: int64(11)
memory usage: 100.5 KB


# Segregated continues & categorical varibale 

In [8]:
prep['attribute4']=prep['attribute4'].astype('category')
prep['attribute5']=prep['attribute5'].astype('category')
prep['device_type']=prep['device_type'].astype('category')
prep['att39']=prep['att39'].astype('category')
prep['att78']=prep['att78'].astype('category')
prep['att2']=prep['att2'].astype('category')
prep['month']=prep['month'].astype('category')

# Scaling continuous variable

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
scaler = StandardScaler()
prep[['att1', 'total_mile','work_days']] = scaler.fit_transform(prep[['att1', 'total_mile','work_days']])
prep.head(5)

,failure,attribute4,attribute5,work_days,device_type,att2,att1,att39,att78,month,total_mile
0,0,52,6,-0.980316,0,7,-0.442297,7,0,1,-0.804348
1,0,0,5,-0.980316,0,0,0.505001,0,0,5,-0.852462
2,0,0,6,-0.980316,0,0,-0.144641,3,0,1,-0.814042
3,0,0,12,-0.570987,0,0,0.399451,0,0,2,-0.253074
4,0,0,6,-0.980316,0,0,0.238975,0,0,1,-0.804375


In [10]:
prep=prep.drop(['failure'],axis=1)

In [11]:
prep.head()

,attribute4,attribute5,work_days,device_type,att2,att1,att39,att78,month,total_mile
0,52,6,-0.980316,0,7,-0.442297,7,0,1,-0.804348
1,0,5,-0.980316,0,0,0.505001,0,0,5,-0.852462
2,0,6,-0.980316,0,0,-0.144641,3,0,1,-0.814042
3,0,12,-0.570987,0,0,0.399451,0,0,2,-0.253074
4,0,6,-0.980316,0,0,0.238975,0,0,1,-0.804375


In [12]:
prep.shape

(1168, 10)

In [13]:
prep_dummy=pd.get_dummies(prep,drop_first=False)

In [14]:
prep_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168 entries, 0 to 1167
Columns: 361 entries, work_days to month_11
dtypes: float64(3), uint8(358)
memory usage: 435.8 KB


In [15]:
prep_dummy.head()

,work_days,att1,total_mile,attribute4_0,attribute4_1,attribute4_2,attribute4_3,attribute4_4,attribute4_5,attribute4_6,...,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11
0,-0.980316,-0.442297,-0.804348,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.980316,0.505001,-0.852462,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,-0.980316,-0.144641,-0.814042,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.570987,0.399451,-0.253074,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,-0.980316,0.238975,-0.804375,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Onehot Encoding Categorical Variable

In [17]:
#cat_data=['attribute4','attribute5','device_type','att2','att39','att78','month']
onehotencoder = OneHotEncoder(categorical_features = [0,1,3,4,6,7,8])
prep_enc = onehotencoder.fit_transform(prep).toarray()
prep_enc

array([[ 0.        ,  0.        ,  0.        , ..., -0.98031558,
        -0.44229736, -0.80434759],
       [ 1.        ,  0.        ,  0.        , ..., -0.98031558,
         0.50500118, -0.85246181],
       [ 1.        ,  0.        ,  0.        , ..., -0.98031558,
        -0.14464102, -0.81404163],
       ..., 
       [ 1.        ,  0.        ,  0.        , ..., -0.22013419,
         0.4568863 , -0.29938106],
       [ 1.        ,  0.        ,  0.        , ..., -0.22013419,
         0.49895879, -0.27459827],
       [ 1.        ,  0.        ,  0.        , ..., -0.22988011,
         0.5251597 , -0.41970768]])

In [18]:
data_scaled = pd.DataFrame(prep_enc,columns= prep_dummy_1.columns)

In [19]:
data_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168 entries, 0 to 1167
Columns: 361 entries, attribute4_0 to total_mile
dtypes: float64(361)
memory usage: 3.2 MB


In [20]:
data_scaled.head()

,attribute4_0,attribute4_1,attribute4_2,attribute4_3,attribute4_4,attribute4_5,attribute4_6,attribute4_7,attribute4_8,attribute4_9,...,month_5,month_6,month_7,month_8,month_9,month_10,month_11,work_days,att1,total_mile
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.980316,-0.442297,-0.804348
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.980316,0.505001,-0.852462
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.980316,-0.144641,-0.814042
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.570987,0.399451,-0.253074
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.980316,0.238975,-0.804375


# test train split before oversampling data

In [21]:
from sklearn.cross_validation import train_test_split as tts
x_train, x_val, y_train, y_val = tts(data_scaled,df['failure'],
                                                  test_size = .2,stratify=df['failure'],
                                                  random_state=42)
x_train.shape,x_val.shape,y_train.shape,y_val.shape

/Users/SKYLER/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


((934, 361), (234, 361), (934,), (234,))

In [22]:
print(y_train.value_counts(normalize=True))
print(y_val.value_counts(normalize=True))

0    0.908994
1    0.091006
Name: failure, dtype: float64
0    0.910256
1    0.089744
Name: failure, dtype: float64


# Oversampling By SMOTE 

In [23]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42, ratio =1.0)
X_train_res, Y_train_res = sm.fit_sample(x_train,y_train)
X_train_res.shape, Y_train_res.shape

/Users/SKYLER/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


((1698, 361), (1698,))

In [24]:
y_train.value_counts(), np.bincount(Y_train_res)

(0    849
 1     85
 Name: failure, dtype: int64, array([849, 849]))

In [25]:
y_val.shape, 
y_val.value_counts()

0    213
1     21
Name: failure, dtype: int64

# Modeling

In [26]:
def Metrics(y_val,pred):
    print('accuray:', accuracy_score(y_val,pred),',recall score:',recall_score(y_val,pred),'\n ConfusionMatrix: \n',
          confusion_matrix(y_val,pred))
    #model_rf.feature_importances_
    print('Precision Score:',precision_score(y_val,pred),'F1_score:',f1_score(y_val,pred))

# Randomforest 

In [27]:
from sklearn.metrics import accuracy_score,recall_score, confusion_matrix, roc_curve, precision_score, f1_score
from sklearn.ensemble import RandomForestClassifier #Random Forest

rf = RandomForestClassifier(n_estimators=145, min_samples_split=15, \
                            max_depth=5,random_state=42)
model_rf = rf.fit(X_train_res,Y_train_res)
pred_rf = model_rf.predict(x_val)
Metrics(y_val,pred_rf)

accuray: 0.961538461538 ,recall score: 0.714285714286 
 ConfusionMatrix: 
 [[210   3]
 [  6  15]]
Precision Score: 0.833333333333 F1_score: 0.769230769231


# Feature Importance - Randomforest

In [40]:
import pandas as pd
feature_importance_rf = pd.DataFrame(rf.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

In [44]:
feature_importance_rf[0:10]

,importance
att2_0,0.125554
attribute4_0,0.111470
att78_0,0.097731
total_mile,0.073053
work_days,0.070639
month_7,0.062459
month_5,0.049563
att1,0.043635
month_3,0.024811
att78_576,0.021014


# Gridsearchcv for Accuracy - with Randomforest

In [30]:
import random
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, 
                            accuracy_score, precision_score, confusion_matrix

pipeline = Pipeline([('clf',RandomForestClassifier(criterion='gini',class_weight ={0:0.3,1:0.7}))])

parameters = {
            'clf__n_estimators':[100,120,140,160],
            'clf__max_depth':[10,15,25,30],
            'clf__min_samples_split':(15,20,25),
           'clf__min_samples_leaf':(1,2)}

grid_search = GridSearchCV(pipeline,parameters,n_jobs=-1,cv=5,scoring='accuracy')
grid_search.fit(X_train_res,Y_train_res)
print ('Best Training score: %0.3f' % grid_search.best_score_)
print ('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print ('\t%s: %r' % (param_name, best_parameters[param_name]))
predictions = grid_search.predict(x_val)
print ("Testing accuracy:",round(accuracy_score(y_val,predictions),4))
print ("recall score:",recall_score(y_val,predictions))
print ("Precision Score:",precision_score(y_val,predictions))
print ("\nComplete report of Testing data\n",classification_report(y_val, predictions))
print ("\n\nRandom Forest Grid Search- Test Confusion Matrix\n\n",pd.crosstab( y_val, predictions,
                                                                rownames = ["Actuall"],colnames = ["Predicted"]))

Best Training score: 0.987
Best parameters set:
	clf__max_depth: 25
	clf__min_samples_leaf: 1
	clf__min_samples_split: 15
	clf__n_estimators: 160
Testing accuracy: 0.9615
recall score: 0.714285714286
Precision Score: 0.833333333333

Complete report of Testing data
              precision    recall  f1-score   support

          0       0.97      0.99      0.98       213
          1       0.83      0.71      0.77        21

avg / total       0.96      0.96      0.96       234



Random Forest Grid Search- Test Confusion Matrix

 Predicted    0   1
Actuall           
0          210   3
1            6  15


# Gridsearchcv for Recall & Precision - with Randomforest

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix


clf = RandomForestClassifier(criterion='gini',n_jobs=-1)

param_grid = {
    'min_samples_split': [3, 5, 10, 15, 20, 25], 
    'n_estimators' : [100,120,140],
    'max_depth': [15,20, 25,30,35],
    'max_features': [3, 5, 10, 20]}

scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score)}

def grid_search_wrapper(refit_score='recall_score'):
    skf = StratifiedKFold(n_splits=10)
    grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score,
                           cv=skf, return_train_score=True, n_jobs=-1,)
    grid_search.fit(X_train_res,Y_train_res)

    # make the predictions
    y_pred = grid_search.predict(x_val)

    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)
    print ("recall score:",recall_score(y_val,y_pred))
    print ("Precision Score:",precision_score(y_val,y_pred))

    # confusion matrix on the test data.
    print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    print(pd.DataFrame(confusion_matrix(y_val, y_pred),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))
    return grid_search

In [34]:
grid_search_clf = grid_search_wrapper(refit_score='recall_score')

Best params for recall_score
{'max_depth': 25, 'max_features': 20, 'min_samples_split': 3, 'n_estimators': 120}
recall score: 0.666666666667
Precision Score: 0.933333333333

Confusion matrix of Random Forest optimized for recall_score on the test data:
     pred_neg  pred_pos
neg       212         1
pos         7        14


# Gradient Boosting

In [35]:
from sklearn.ensemble import GradientBoostingClassifier 
g_clf = GradientBoostingClassifier(n_estimators=700,learning_rate=0.5,max_depth=15)
g_clf.fit(X_train_res,Y_train_res)
pred_g=g_clf.predict(x_val)
Metrics(y_val,pred_g)

accuray: 0.961538461538 ,recall score: 0.809523809524 
 ConfusionMatrix: 
 [[208   5]
 [  4  17]]
Precision Score: 0.772727272727 F1_score: 0.790697674419


# Feature Importance - GradientBoosting

In [46]:
import pandas as pd
feature_importances_gb = pd.DataFrame(g_clf.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

In [51]:
feature_importances_gb[0:10]

,importance
att78_0,0.005212
work_days,0.005166
att2_0,0.003204
att1,0.002267
month_7,0.001733
month_5,0.001570
total_mile,0.000872
month_6,0.000591
month_2,0.000364
attribute4_0,0.000363
